# Decrypting a Vignere Cipher

Below is code that will transcribe any ciphertext which was encrypted using a Vignere Cipher into plaintext. <br>
<br>
A Vignere Cipher is a variation of a shift cipher. The key for the encryption is a vector of numbers of the form $(s_1,s_2,...,s_n)$. Both the key and the length of the key $n$ is unknown for unintended receivers. Encryption is achieved by shifting the first letter of the ciphertext by an amount $s_1$, shifting the second letter by an amount $s_2$, and so on. Once we reach the $(n+1)^{\textrm{th}}$ entry of the ciphertext, we restart from the first element of the key ($s_1$). <br>
<br>
A ciphertext attack of the system relies on the fact that not all letters of the english alphabet occur with equal frequency in written text. Refer to the second chapter of the textbook 'Introduction to Cryptography with Coding Theory' by Trappe and Washington for an explanation of why the approach used in the code below would work.

In [1]:
import numpy as np


alphabet = list('abcdefghijklmnopqrstuvwxyz')

#Frequency of occurrence of each letter of the alphabet, obtained from Trappe and Washington
ori_freq = [0.082,0.015,0.028,0.043,0.127,0.022,0.020,0.061,0.070,0.002,0.008,0.040,0.024,0.067,0.075,0.019,0.001,0.060,0.063,0.091,0.028,0.01,0.023,0.001,0.02,0.001]

## Obtaining the key length



In [3]:
def key_length(ciphertext,max_len):
    ciphertext = list(ciphertext)
    coincidences = np.zeros(max_len)
    shift_len = 1
    while shift_len < max_len:
        shifted_text = np.roll(ciphertext,shift_len)
        for ori,new in zip(ciphertext,shifted_text):
            if ori == new:
                coincidences[shift_len] += 1
        shift_len += 1
    
    #Get 5 largest coincidences
    most_coincidences = sorted(np.unique(np.array(coincidences)),reverse=True)[0:5]
    
    out_key_lens = []
    out_coincidences = []
    for i in range(0,5):
        possible_key_lens = np.where(np.array(coincidences)==most_coincidences[i])[0]
        for possible_key_len in possible_key_lens:
            out_key_lens.append(possible_key_len)
            out_coincidences.append(coincidences[possible_key_len])

    return out_key_lens, out_coincidences

## Obtaining the key once the key length is known

In [4]:
def dot_product(ciphertext,key_len,exp_freq,start):
    obs_freq = []
    new_list = []
    ciphertext = list(ciphertext)
    for index in range(start,len(ciphertext),key_len):
            new_list.append(ciphertext[index])
    
    for i in range(0,26):
        obs_freq.append(new_list.count(alphabet[i])/len(new_list))
    
    dot = 0
    for i in range(0,26):
        dot += obs_freq[i]*exp_freq[i]
    
    return dot

In [5]:
def get_key_vals(ciphertext,key_len,start):
    
    dots = []
    for index in range(1,26):
        exp_freq = list(np.roll(ori_freq,index))
        dots.append(dot_product(ciphertext,key_len,exp_freq,start))
    
    #print(dots)
    return (dots.index(max(dots)))

In [49]:
def get_key(ciphertext,key_len):
    
    key = []
    for i in range(0,key_len):
        key.append(get_key_vals(ciphertext,key_len,i))
    
    alphabet_key = []
    for index in key:
        alphabet_key.append(alphabet[index+1])
    
    return key, alphabet_key

## Decrypting a given ciphertext

We can now combine all the above functions into a single function that will convert a ciphertext encrypted using a Vignere cipher into plaintext.

In [2]:
def shift(letter,amount):
    out_list = []
    ori_index = alphabet.index(letter)
    new_index = (ori_index - amount)%26

    return alphabet[new_index-1]

In [60]:
def decrypt_vignere(ciphertext,max_key_len):
    
    key_len = key_length(ciphertext,max_key_len)[0][0]
    
    key = get_key(ciphertext,key_len)[0]
    
    ciphertext = list(ciphertext)
    
    plaintext = []
    for i in range(0,len(ciphertext)):
        plaintext.append(shift(ciphertext[i],key[i % key_len]))
    
    return(''.join(plaintext))

## Examples

### Example 1

Problem 7 from section 2.14 of Introduction to Cryptography with Coding Theory by Trappe and Washington

In [51]:
ciphertext1 = 'hdsfgvmkoowafweetcmfthskucaqbilgjofmaqlgspvatvxqbiryscpcfrmvswrvnqlszdmgaoqsakmlupsqforvtwvdfcjzvgsoaoqsacjkbrsevbelvbksarlscdcaarmnvrysywxqgvellcyluwwveoafgclazowafojdlhssfiksepsoywxafowlbfcsocylngqsyzxgjbmlvgrggokgfgmhlmejabsjvgmlnrvqzcrggcrghgeupcyfgtydycjkhqluhgxgzovqswpdvbwsffsenbxapasgazmyuhgsfhmftayjxmwznrsofrsoaopgauaaarmftqsmahvqecev'

In [53]:
key_length(ciphertext1,6)

([4, 3, 2, 5, 1, 0], [25.0, 15.0, 14.0, 14.0, 11.0, 0.0])

Thus, it appears that the most likely key length is 4. Using this, we can try to figure out what the key is.

In [54]:
get_key(ciphertext1,4)

([12, 13, 3, 17], ['n', 'o', 'e', 's'])

In [61]:
decrypt_vignere(ciphertext1,6)

'uponthisbasisiamgoingtoshowyouhowabunchofbrightyoungfolksdidfindachampionamanwithboysandgirlsofhisownamanofsodominatingandhappyindividualitythatyouthisdrawntohimasisaflytoasugarbowlitisastoryaboutasmalltownitisnotagossipyyarnnorisitadrymonotonousaccountfullofsuchcustomaryfillinsasromanticmoonlightcastingmurkyshadowsdownalongwindingcountryroad'

### Example 2

Problem 8 from section 2.14 of Introduction to Cryptography with Coding Theory by Trappe and Washington

In [56]:
ciphertext2 = 'ocwyikoooniwugpmxwktzdwgtssayjzwyemdlbnqaaavsuwdvbrflauplooubfgqhgcscmgzlatoedcsdeidpbhtmuovpiekifpimfnoamvlpqfxejsmxmpgkccaykwfzpyuavtelwhrhmwkbbvgtguvtefjlodfefkvpxsgrsorvgtajbsauhzrzalkwuowhgedefnswmrciwcpaaavogpdnfpktdbalsisurlnpsjyeatcuceesohhdarkhwotikbroqrdfmzghgucebvgwcdqxgpbgqwlpbdaylooqdmuhbdqgmyweuik'

In [57]:
key_length(ciphertext2,26)

([6, 12, 18, 5, 15, 20], [25.0, 19.0, 16.0, 15.0, 15.0, 14.0])

In [58]:
get_key(ciphertext2,6)

([6, 13, 10, 11, 3, 17], ['h', 'o', 'l', 'm', 'e', 's'])

In [59]:
decrypt_vignere(ciphertext2,12)

'holmeshadbeenseatedforsomehoursinsilencewithhislongthinbackcurvedoverachemicalvesselinwhichhewasbrewingaparticularlymalodorousproducthisheadwassunkuponhisbreastandhelookedfrommypointofviewlikeastrangelankbirdwithdullgreyplumageandablacktopknotsowatsonsaidhesuddenlyyoudonotproposetoinvestinsouthafricansecurities'